In [2]:
!pip install transformers sentencepiece torchtext --quiet
!pip install sacrebleu --quiet



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.2 MB/s eta 0:00:00


In [3]:
import os
import shutil
import zipfile
from urllib.request import urlretrieve

# Dataset URL
url = "https://github.com/amir9ume/urdu_ghazals_rekhta/archive/refs/heads/master.zip"
zip_path = "database.zip"  # Save as database.zip
extract_path = "dataset"   # Final folder

# Remove existing zip and folder if they exist
if os.path.exists(zip_path):
    os.remove(zip_path)
if os.path.exists(extract_path):
    shutil.rmtree(extract_path)

# Download dataset
print("Downloading dataset...")
urlretrieve(url, zip_path)
print("Download completed.")

# Extract the zip file to a temporary folder
temp_extract_path = "temp_extracted"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_extract_path)
print(f"Extracted to {temp_extract_path}/")

# Automatically detect the folder inside the zip
inner_folders = [f for f in os.listdir(temp_extract_path) if os.path.isdir(os.path.join(temp_extract_path, f))]
if len(inner_folders) == 0:
    raise FileNotFoundError("No folder found inside the extracted zip!")
inner_folder_path = os.path.join(temp_extract_path, inner_folders[0])

# Move contents to 'dataset' folder
shutil.move(inner_folder_path, extract_path)

# Remove temporary folder
shutil.rmtree(temp_extract_path)

# List files in 'dataset'
print("Files in dataset folder:")
files = os.listdir(extract_path)
print(files)

# Example: filter dataset files
dataset_files = [f for f in files if f.endswith(('.csv', '.txt'))]
print("Dataset files available:", dataset_files)

# Read the first dataset file with Pandas
import pandas as pd
if dataset_files:
    df = pd.read_csv(os.path.join(extract_path, dataset_files[0]))
    print("Sample data:")
    print(df.head())


Download completed.
Extracted to temp_extracted/
Files in dataset folder:
['dataset', 'README.md', 'sample_dataset', 'LICENSE', 'rekhta_parser.ipynb']
Dataset files available: []


In [4]:
import os
import zipfile
from urllib.request import urlretrieve

# Dataset URL
url = "https://github.com/amir9ume/urdu_ghazals_rekhta/archive/refs/heads/master.zip"
zip_path = "dataset.zip"
extract_path = "dataset"

# Download dataset if not already present
if not os.path.exists(zip_path):
    print("Downloading dataset...")
    urlretrieve(url, zip_path)
    print("Download completed.")

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"Extracted to {extract_path}/")

# Automatically detect the folder inside the extracted zip
extracted_contents = os.listdir(extract_path)
inner_folders = [f for f in extracted_contents if os.path.isdir(os.path.join(extract_path, f))]
if len(inner_folders) == 0:
    raise FileNotFoundError("No folder found inside the extracted zip!")
extracted_folder = os.path.join(extract_path, inner_folders[0])

# List top-level files and folders
print("Files and folders in extracted folder:")
print(os.listdir(extracted_folder))

# Recursively list all dataset files (CSV/TXT)
dataset_files = []
for root, dirs, files in os.walk(extracted_folder):
    for f in files:
        if f.endswith(('.csv', '.txt')):
            dataset_files.append(os.path.join(root, f))

print("\nAll dataset files found:")
for f in dataset_files:
    print(f)


Download completed.
Extracted to dataset/
Files and folders in extracted folder:
['dataset', 'README.md', 'sample_dataset', 'LICENSE', 'rekhta_parser.ipynb']

All dataset files found:


In [5]:



!pwd
!ls -l


/content
total 3992
-rw-r--r-- 1 root root 2037669 Sep 26 12:03 database.zip
drwxr-xr-x 5 root root    4096 Sep 26 12:03 dataset
-rw-r--r-- 1 root root 2037669 Sep 26 12:03 dataset.zip
drwxr-xr-x 1 root root    4096 Sep 24 13:41 sample_data


In [6]:
!apt-get install -y git
!git clone https://github.com/amir9ume/urdu_ghazals_rekhta.git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Cloning into 'urdu_ghazals_rekhta'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 112 (delta 7), reused 6 (delta 6), pack-reused 103 (from 1)
Receiving objects: 100% (112/112), 2.03 MiB | 25.71 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [ ]:
import os

# Define the path to the dataset folder
dataset_path = '/content/urdu_ghazals_rekhta/dataset'

# Check if the dataset folder exists
if os.path.exists(dataset_path):
    print(f"Dataset folder found at: {dataset_path}")
else:
    print("Dataset folder not found.")


In [11]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [12]:
import os

print("MyDrive contents:", os.listdir("/content/drive/MyDrive"))
print("Dataset contents:", os.listdir("/content/drive/MyDrive/dataset")[:10])


MyDrive contents: ['Colab Notebooks', 'dataset']
Dataset contents: ['.DS_Store', 'sahir-ludhianvi', 'parveen-shakir', 'noon-meem-rashid', 'waseem-barelvi', 'mohsin-naqvi', 'naji-shakir', 'nida-fazli', 'nazm-tabatabai', 'wali-mohammad-wali']


In [13]:
import os
import pandas as pd
import unicodedata
from sklearn.model_selection import train_test_split

# -------- Urdu normalization helpers --------
def normalize_urdu(text: str) -> str:
    text = unicodedata.normalize("NFC", text)
    diacritics = [
        "\u064B", "\u064C", "\u064D", "\u064E", "\u064F",
        "\u0650", "\u0651", "\u0652"
    ]
    for d in diacritics:
        text = text.replace(d, "")
    text = text.replace("\u200c", "").replace("\u200d", "")
    return text.strip()

def normalize_roman(text: str) -> str:
    return text.strip().lower()

# -------- Main builder --------
def build_and_split_dataset(root_dir, output_dir="."):
    urdu_texts, roman_texts = [], []

    for poet in os.listdir(root_dir):
        poet_path = os.path.join(root_dir, poet)
        if not os.path.isdir(poet_path):
            continue

        urdu_dir = os.path.join(poet_path, "ur")
        roman_dir = os.path.join(poet_path, "en")

        if not os.path.exists(urdu_dir) or not os.path.exists(roman_dir):
            print(f"⚠️ Skipping {poet}: missing ur/en folders")
            continue

        for filename in os.listdir(urdu_dir):
            urdu_file = os.path.join(urdu_dir, filename)
            roman_file = os.path.join(roman_dir, filename)

            if not os.path.exists(roman_file):
                print(f"⚠️ Skipping {filename}: no matching Roman file")
                continue

            try:
                with open(urdu_file, "r", encoding="utf-8", errors="ignore") as f_ur, \
                     open(roman_file, "r", encoding="utf-8", errors="ignore") as f_ro:
                    urdu_lines = [line.strip() for line in f_ur if line.strip()]
                    roman_lines = [line.strip() for line in f_ro if line.strip()]

                if len(urdu_lines) != len(roman_lines):
                    print(f"⏩ Skipping {filename}: Urdu={len(urdu_lines)} vs Roman={len(roman_lines)}")
                    continue

                urdu_texts.extend(urdu_lines)
                roman_texts.extend(roman_lines)
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                continue


    # Build DataFrame
    df = pd.DataFrame({"urdu": urdu_texts, "roman": roman_texts})

    # Ensure columns are string type and handle NaN before cleaning/filtering
    df["urdu"] = df["urdu"].astype(str).fillna("")
    df["roman"] = df["roman"].astype(str).fillna("")

    # Apply cleaning functions
    df["urdu"] = df["urdu"].apply(normalize_urdu)
    df["roman"] = df["roman"].apply(normalize_roman)


    # Filter out empty rows
    df = df[(df["urdu"] != "") & (df["roman"] != "")]


    os.makedirs(output_dir, exist_ok=True)
    dataset_path = os.path.join(output_dir, "dataset.csv")
    df.to_csv(dataset_path, index=False, encoding="utf-8-sig")
    print(f"✅ Dataset built with {len(df)} aligned pairs → saved to {dataset_path}")

    # Split
    if len(df) < 4:
        print("⚠️ Not enough samples to split. Skipping splitting step.")
        return

    train, temp = train_test_split(df, test_size=0.5, random_state=42)
    valid, test = train_test_split(temp, test_size=0.5, random_state=42)

    train.to_csv(os.path.join(output_dir, "train.csv"), index=False, encoding="utf-8-sig")
    valid.to_csv(os.path.join(output_dir, "valid.csv"), index=False, encoding="utf-8-sig")
    test.to_csv(os.path.join(output_dir, "test.csv"), index=False, encoding="utf-8-sig")

    print(f"📂 Train: {len(train)} rows")
    print(f"📂 Valid: {len(valid)} rows")
    print(f"📂 Test:  {len(test)} rows")

# Run
build_and_split_dataset(
    "/content/drive/MyDrive/dataset",   # input dataset folder (where ur/en are stored per poet)
    "/content/drive/MyDrive/NLP"        # output folder (will be created)
)

⏩ Skipping gar-tujh-ko-hai-yaqiin-e-ijaabat-duaa-na-maang-mirza-ghalib-ghazals: Urdu=18 vs Roman=17
⏩ Skipping jis-zakhm-kii-ho-saktii-ho-tadbiir-rafuu-kii-mirza-ghalib-ghazals: Urdu=14 vs Roman=12
⏩ Skipping vaan-pahunch-kar-jo-gash-aataa-pae-ham-hai-ham-ko-mirza-ghalib-ghazals: Urdu=28 vs Roman=24
⏩ Skipping gar-na-andoh-e-shab-e-furqat-bayaan-ho-jaaegaa-mirza-ghalib-ghazals: Urdu=22 vs Roman=18
⏩ Skipping khamoshiyon-men-tamaashaa-adaa-nikaltii-hai-mirza-ghalib-ghazals: Urdu=12 vs Roman=8
⏩ Skipping jab-ba-taqriib-e-safar-yaar-ne-mahmil-baandhaa-mirza-ghalib-ghazals: Urdu=24 vs Roman=20
⏩ Skipping shumaar-e-subha-marguub-e-but-e-mushkil-pasand-aayaa-mirza-ghalib-ghazals-1: Urdu=14 vs Roman=10
⏩ Skipping koh-ke-hon-baar-e-khaatir-gar-sadaa-ho-jaaiye-mirza-ghalib-ghazals: Urdu=12 vs Roman=6
⏩ Skipping shab-ki-vo-majlis-faroz-e-khalvat-e-naamuus-thaa-mirza-ghalib-ghazals: Urdu=14 vs Roman=8
⏩ Skipping hujuum-e-naala-hairat-aajiz-e-arz-e-yak-afgaan-hai-mirza-ghalib-ghazals: Urdu=22 vs R

In [23]:
import os
import pandas as pd

# Paths
data_dir = "/content/drive/MyDrive/NLP"
train_csv = os.path.join(data_dir, "train.csv")
valid_csv = os.path.join(data_dir, "valid.csv")
test_csv  = os.path.join(data_dir, "test.csv")

# Load data
train_df = pd.read_csv(train_csv)
valid_df = pd.read_csv(valid_csv)
test_df  = pd.read_csv(test_csv)

# Build joint character vocabulary (Urdu + Roman)
all_text = "".join(train_df['urdu'].astype(str).tolist() + train_df['roman'].astype(str).tolist())
chars = sorted(list(set(all_text)))
stoi = {ch: i+4 for i, ch in enumerate(chars)}   # +4 to reserve special tokens
itos = {i: ch for ch, i in stoi.items()}

# special tokens
PAD, UNK, BOS, EOS = 0, 1, 2, 3
stoi.update({"<pad>": PAD, "<unk>": UNK, "<s>": BOS, "</s>": EOS})
itos.update({PAD: "<pad>", UNK: "<unk>", BOS: "<s>", EOS: "</s>"})

vocab_size = len(stoi)
print("✅ Vocab size:", vocab_size)


✅ Vocab size: 94


In [24]:
import torch
from torch.utils.data import Dataset, DataLoader

class UrduRomanDataset(Dataset):
    def __init__(self, df, stoi, max_len=128, src_col='urdu', tgt_col='roman'):
        self.stoi = stoi
        self.src = df[src_col].astype(str).tolist()
        self.tgt = df[tgt_col].astype(str).tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.src)

    def encode(self, text):
        ids = [self.stoi.get(ch, 1) for ch in text]  # 1 = <unk>
        ids = [2] + ids + [3]   # add <s>, </s>
        return ids[:self.max_len]

    def __getitem__(self, idx):
        src_ids = self.encode(self.src[idx])
        tgt_ids = self.encode(self.tgt[idx])
        return {
            'src_ids': torch.tensor(src_ids, dtype=torch.long),
            'tgt_ids': torch.tensor(tgt_ids, dtype=torch.long)
        }

def collate_fn(batch, pad_id=0):
    srcs = [b['src_ids'] for b in batch]
    tgts = [b['tgt_ids'] for b in batch]
    src_pad = torch.nn.utils.rnn.pad_sequence(srcs, batch_first=True, padding_value=pad_id)
    tgt_pad = torch.nn.utils.rnn.pad_sequence(tgts, batch_first=True, padding_value=pad_id)
    return src_pad, tgt_pad

# Loaders
train_ds = UrduRomanDataset(train_df, stoi, max_len=128)
valid_ds = UrduRomanDataset(valid_df, stoi, max_len=128)
test_ds  = UrduRomanDataset(test_df,  stoi, max_len=128)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_ds, batch_size=64, shuffle=False, collate_fn=collate_fn)


In [25]:
import torch.nn as nn
import torch

class EncoderBiLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, enc_hidden, enc_layers=2, dropout=0.3, pad_id=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_id)
        self.rnn = nn.LSTM(emb_dim, enc_hidden, num_layers=enc_layers,
                           dropout=dropout, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        emb = self.dropout(self.embedding(x))
        outputs, (h, c) = self.rnn(emb)
        return outputs, (h, c)

class Attention(nn.Module):
    def __init__(self, enc_hidden, dec_hidden):
        super().__init__()
        self.attn = nn.Linear(enc_hidden*2 + dec_hidden, dec_hidden)
        self.v = nn.Linear(dec_hidden, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        scores = self.v(energy).squeeze(2)
        attn_weights = torch.softmax(scores, dim=1)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attn_weights

class DecoderLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, enc_hidden, dec_hidden, dec_layers=2, dropout=0.3, pad_id=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_id)
        self.rnn = nn.LSTM(emb_dim + enc_hidden*2, dec_hidden, num_layers=dec_layers,
                           dropout=dropout, batch_first=True)
        self.attention = Attention(enc_hidden, dec_hidden)
        self.fc_out = nn.Linear(dec_hidden + enc_hidden*2 + emb_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_step, hidden, encoder_outputs):
        emb = self.dropout(self.embedding(input_step))
        top_hidden = hidden[0][-1]
        context, _ = self.attention(top_hidden, encoder_outputs)
        context = context.unsqueeze(1)
        rnn_input = torch.cat((emb, context), dim=2)
        output, hidden = self.rnn(rnn_input, hidden)
        output = output.squeeze(1)
        emb_s = emb.squeeze(1)
        concat = torch.cat((output, context.squeeze(1), emb_s), dim=1)
        preds = self.fc_out(concat)
        return preds, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        batch_size, max_len = tgt.size()
        vocab_size = self.decoder.fc_out.out_features
        encoder_outputs, _ = self.encoder(src)
        dec_h = torch.zeros(self.decoder.rnn.num_layers, batch_size, self.decoder.rnn.hidden_size).to(self.device)
        dec_c = torch.zeros_like(dec_h)
        outputs = torch.zeros(batch_size, max_len, vocab_size).to(self.device)
        input_step = tgt[:,0].unsqueeze(1)  # BOS
        for t in range(1, max_len):
            preds, (dec_h, dec_c) = self.decoder(input_step, (dec_h, dec_c), encoder_outputs)
            outputs[:,t,:] = preds
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = preds.argmax(1).unsqueeze(1)
            input_step = tgt[:,t].unsqueeze(1) if teacher_force else top1
        return outputs


In [26]:
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

emb_dim = 128
enc_hidden = 256
dec_hidden = 256

encoder = EncoderBiLSTM(vocab_size, emb_dim, enc_hidden).to(device)
decoder = DecoderLSTM(vocab_size, emb_dim, enc_hidden, dec_hidden).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=PAD)

def train_epoch(model, loader, optimizer, criterion, clip=1.0):
    model.train()
    total_loss = 0
    for src, tgt in tqdm(loader):
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()
        output = model(src, tgt)
        out = output[:,1:,:].reshape(-1, output.shape[-1])
        target = tgt[:,1:].reshape(-1)
        loss = criterion(out, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for src, tgt in loader:
            src, tgt = src.to(device), tgt.to(device)
            output = model(src, tgt, teacher_forcing_ratio=0.0)
            out = output[:,1:,:].reshape(-1, output.shape[-1])
            target = tgt[:,1:].reshape(-1)
            loss = criterion(out, target)
            total_loss += loss.item()
    return total_loss / len(loader)

best_valid = float("inf")
for epoch in range(1, 21):
    tr_loss = train_epoch(model, train_loader, optimizer, criterion)
    val_loss = evaluate(model, valid_loader, criterion)
    print(f"Epoch {epoch} | Train {tr_loss:.3f} | Valid {val_loss:.3f}")
    if val_loss < best_valid:
        best_valid = val_loss
        torch.save(model.state_dict(), f"{data_dir}/best_char_seq2seq.pth")
        print("✅ Saved best model")


100%|██████████| 163/163 [00:35<00:00,  4.56it/s]


Epoch 1 | Train 2.900 | Valid 3.078
✅ Saved best model


100%|██████████| 163/163 [00:37<00:00,  4.36it/s]


Epoch 2 | Train 2.192 | Valid 2.355
✅ Saved best model


100%|██████████| 163/163 [00:35<00:00,  4.63it/s]


...

Epoch 18 | Train 0.156 | Valid 1.346


100%|██████████| 163/163 [00:34<00:00,  4.73it/s]


Epoch 19 | Train 0.152 | Valid 1.343


100%|██████████| 163/163 [00:34<00:00,  4.76it/s]


Epoch 20 | Train 0.161 | Valid 1.339


In [29]:
!pip install python-Levenshtein --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.6 MB/s eta 0:00:00


In [30]:
from sacrebleu import corpus_bleu
import Levenshtein as Lev

def decode_ids(ids):
    tokens = [itos.get(int(x), "") for x in ids if int(x) not in (PAD,BOS)]
    return "".join(tokens).replace("</s>", "")

def translate_sentence(model, sentence, max_len=128):
    model.eval()
    ids = [stoi.get(ch, UNK) for ch in sentence]
    ids = [BOS] + ids + [EOS]
    src = torch.tensor([ids], dtype=torch.long).to(device)
    with torch.no_grad():
        encoder_outputs, _ = model.encoder(src)
        dec_h = torch.zeros(model.decoder.rnn.num_layers, 1, model.decoder.rnn.hidden_size).to(device)
        dec_c = torch.zeros_like(dec_h)
        input_step = torch.tensor([[BOS]], dtype=torch.long).to(device)
        output_ids = []
        for _ in range(max_len):
            preds, (dec_h, dec_c) = model.decoder(input_step, (dec_h, dec_c), encoder_outputs)
            top1 = preds.argmax(1).item()
            if top1 == EOS:
                break
            output_ids.append(top1)
            input_step = torch.tensor([[top1]], dtype=torch.long).to(device)
    return decode_ids(output_ids)

# Evaluate on test set
refs, hyps = [], []
for i in range(200):  # subset
    src = test_df.iloc[i]['urdu']
    ref = test_df.iloc[i]['roman']
    hyp = translate_sentence(model, src)
    refs.append(ref)
    hyps.append(hyp)

bleu = corpus_bleu(hyps, [refs])
print("BLEU:", bleu.score)

cer = sum(Lev.distance(r, h)/max(1,len(r)) for r,h in zip(refs,hyps)) / len(refs)
print("Avg CER:", cer)


BLEU: 76.78728113761596
Avg CER: 0.0428140260875769


In [31]:
src_sentence = "محبت سب سے خوبصورت جذبہ ہے"
roman_output = translate_sentence(model, src_sentence)
print("Urdu :", src_sentence)
print("Roman:", roman_output)


Urdu : محبت سب سے خوبصورت جذبہ ہے
Roman: mohabbat sab se ḳhūbas-ūrat-e-jazba hai


In [32]:
src_sentence = "دوستی دلوں کو جوڑتی ہے"
roman_output = translate_sentence(model, src_sentence)
print("Urdu :", src_sentence)
print("Roman:", roman_output)


Urdu : دوستی دلوں کو جوڑتی ہے
Roman: dostī diloñ ko joḍtī hai


In [33]:
src_sentence = "وقت کبھی رکتا نہیں"
roman_output = translate_sentence(model, src_sentence)
print("Urdu :", src_sentence)
print("Roman:", roman_output)


Urdu : وقت کبھی رکتا نہیں
Roman: vaqt kabhī ruktā nahīñ


In [34]:
src_sentence = "خواب حقیقت بن سکتے ہیں"
roman_output = translate_sentence(model, src_sentence)
print("Urdu :", src_sentence)
print("Roman:", roman_output)


Urdu : خواب حقیقت بن سکتے ہیں
Roman: ḳhvāb-e-haqīqat ban sakte haiñ


In [35]:
# Load a custom Urdu input
src_sentence = "زندگی ایک خواب ہے"

# Translate
roman_output = translate_sentence(model, src_sentence)
print("Urdu :", src_sentence)
print("Roman:", roman_output)


Urdu : زندگی ایک خواب ہے
Roman: zindagī ek ḳhvāb hai
